In [3]:
pip install bs4


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [7]:
import requests 
from bs4 import BeautifulSoup
import time

mainUrl = 'https://www.4icu.org/de/universities/'
response = requests.get(mainUrl)
response.encoding = 'utf-8'
soup = BeautifulSoup(response.text,'lxml')



In [8]:
tables = soup.find('table', class_ = 'table')
stateLinks = []
baseUrl = 'https://www.4icu.org'

for aTag in tables.find_all('a', href=True):
    stateName = aTag.text.strip()
    stateUrl = baseUrl + aTag['href']
    stateLinks.append({'state name':stateName,'state url':stateUrl})

for state in stateLinks:
    print(f"state: {state['state name']}, URL: {state['state url']}")

print(f"Total {len(stateLinks)} states")

state: Baden-Wurttemberg, URL: https://www.4icu.org/de/baden-wurttemberg/
state: Bavaria, URL: https://www.4icu.org/de/bavaria/
state: Berlin, URL: https://www.4icu.org/de/berlin/
state: Brandenburg, URL: https://www.4icu.org/de/brandenburg/
state: Bremen, URL: https://www.4icu.org/de/bremen/
state: Hamburg, URL: https://www.4icu.org/de/hamburg/
state: Hesse, URL: https://www.4icu.org/de/hesse/
state: Lower Saxony, URL: https://www.4icu.org/de/lower-saxony/
state: Mecklenburg-Vorpommern, URL: https://www.4icu.org/de/mecklenburg-vorpommern/
state: North Rhine-Westphalia, URL: https://www.4icu.org/de/north-rhine-westphalia/
state: Rhineland-Palatinate, URL: https://www.4icu.org/de/rhineland-palatinate/
state: Saarland, URL: https://www.4icu.org/de/saarland/
state: Saxony, URL: https://www.4icu.org/de/saxony/
state: Saxony-Anhalt, URL: https://www.4icu.org/de/saxony-anhalt/
state: Schleswig-Holstein, URL: https://www.4icu.org/de/schleswig-holstein/
state: Thuringia, URL: https://www.4icu.

In [9]:
stateUniversities = []
for state in stateLinks:
    url = state['state url']
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'lxml')
    uni_table = soup.find('tbody')
    universityLinks = []
    
    for aTag in uni_table.find_all('a', href=True):
        if aTag['href'] == '/about/add.htm':
            continue
        uniUrl = baseUrl + aTag['href']
        universityLinks.append(uniUrl)
    stateUniversities.append({'state':state['state name'], 'universityLinks':universityLinks})


    
        
    



In [10]:
import json
universities = []
for university in stateUniversities:
    stateName = university['state']
    for uniUrl in university['universityLinks']:
        response = requests.get(uniUrl)
        response.encoding = 'utf-8'
        soup = BeautifulSoup(response.text, 'lxml')
        logo = soup.find('img', attrs={"itemprop":"logo"})
        uniName = soup.find('h1', attrs={"itemprop":"name"})
        cityName = soup.find('span', attrs={"itemprop":"addressLocality"})
        type = soup.find('p' ,class_ = 'lead').find('strong')
        foundedYear = soup.find('table', class_='table borderless').find('span', attrs={"itemprop":"foundingDate"})
        socialLinks = soup.find('div', attrs={"id":"social-media"}).find_all('a',attrs={"itemprop":"sameAs"})
        uniLink = soup.find('a', attrs={"itemprop":"url"})
        def determine_media_type(url):
            if 'facebook.com' in url:
                return 'facebook'
            elif 'instagram.com' in url:
                return 'instagram'
            elif 'twitter.com' in url:
                return 'twitter'
            elif 'linkedin.com' in url:
                return 'linkedin'
            elif 'youtube.com' in url:
                return 'youtube'
            else:
                return 'unknown'
        socialUrls = []
        for url in socialLinks:
            socialUrls.append({'media': determine_media_type(url['href']),'link':url['href']})
        social_media_map = {
        'facebook': '',
        'twitter': '',
        'instagram': '',
        'linkedin': '',
        'youtube': ''
        }
        for social_url in socialUrls:
            url = social_url['link']
            if 'facebook.com' in url:
                social_media_map['facebook'] = url
            elif 'twitter.com' in url:
                social_media_map['twitter'] = url
            elif 'instagram.com' in url:
                social_media_map['instagram'] = url
            elif 'linkedin.com' in url:
                social_media_map['linkedin'] = url
            elif 'youtube.com' in url:
                social_media_map['youtube'] = url
        entry = {
            "name":uniName.text.strip(),
            "location":{
                "country":"Germany",
                "state":stateName,
                "city":cityName.text.strip()
            },
            "logoSrc":logo['src'],
            "type":type.text.strip(),
            "establishedYear":foundedYear.text.strip(),
            "contact": {
     "facebook": social_media_map['facebook'],
    "twitter": social_media_map['twitter'],
    "instagram": social_media_map['instagram'],
    "officialWebsite": str(uniLink['href']),
    "linkedin": social_media_map['linkedin'],
    "youtube": social_media_map['youtube']
            }
        }
        universities.append(json.dumps(entry, ensure_ascii=False))


        

In [11]:
with open('universities.json', 'w', encoding='utf-8') as f:
    json.dump(universities, f, ensure_ascii=False, indent=4)